### 2.2.3 標準インターフェイスとしての列


#### 概要

##### 列の演算
あるプログラムで処理の共通パターンを見出し、共通化します。
共通モジュールとして以下を挙げます。
- 列挙(enumerator) ※こちらはプログラム毎に異なります。
- フィルタ(filter)
- マップ(map)
- 集積器 (accumulator)

これらを使って、完結に処理を記述する方法を紹介します。  
各モジュールのインターフェースを入力・出力にリストを使用することで、  
各モジュールが接続しやすくなります。  
（リストを標準インターフェースと言っています）

##### マップのネスト
ネストしたループをマップのネストで実現します。

##### 練習問題
- [練習問題2.33 accumulateを使ったmap/append/lengthの実装](../exercises/2.33.ipynb)
- [練習問題2.34 ホーナー法](../exercises/2.34.ipynb)
- [練習問題2.35 count-leaves](../exercises/2.35.ipynb)
- [練習問題2.36 accumulate-n](../exercises/2.36.ipynb)
- [練習問題2.37 行列演算](../exercises/2.37.ipynb)
- [練習問題2.38 fold-rightとfold-left](../exercises/2.38.ipynb)
- [練習問題2.39 fold-rightとfold-leftを使ったreverseの実装](../exercises/2.39.ipynb)
- [練習問題2.40 unique-pairs/prime-sum-pairs](../exercises/2.40.ipynb)
- [練習問題2.41 3変数のマップ](../exercises/2.41.ipynb)
- [練習問題2.42 8クイーンパズル](../exercises/2.42.ipynb)
- [練習問題2.43 間違った8クイーンパズルの回答](../exercises/2.43.ipynb)

#### はじめに

ここでは2つのプログラムについて考えます。

- sum-odd-squares手続き・・・⽊を引数に取り、奇数の葉の⼆乗の合計を出力する。
- even-fibs手続き・・・フィボナッチ数が偶数となる値のリストを出力する。

In [1]:
; ⽊を引数に取り、奇数の葉の⼆乗の合計を出力する。
(define (square x)(* x x))

(define (sum-odd-squares tree)
  (cond ((null? tree) 0)
        ((not (pair? tree))  ; 葉に到達したら
         (if (odd? tree) (square tree) ; 葉の値が奇数なら2乗を計算する
             0)                        ; 葉の値が偶数なら無視する
         )
        (else (+ (sum-odd-squares (car tree)) (sum-odd-squares (cdr tree)))))
  )
(define x (list 1 (list 2 (list 3 4) 5) (list 6 7)))
(display x)
(newline)
(display (sum-odd-squares x))
(newline)

(1 (2 (3 4) 5) (6 7))
84


In [2]:
; フィボナッチ数が偶数となる値のリストを出力する。
(define (fib n)
  (cond ((= n 0) 0)
        ((= n 1) 1) 
        (else (+ (fib (- n 1)) (fib (- n 2))))
        )
  )

(define (even-fibs n)
  (define (next k)
    (if (> k n) '()
        (let ((f (fib k)))
          (if (even? f) (cons f (next (+ k 1))) ; 偶数のフィボナッチ数は加算の対象とする -> consでつなぎ合わせてリストにする
              (next (+ k 1))))                  ; 奇数のフィボナッチ数は無視する
      )
    )
  (next 0)
  )

(display (even-fibs 10))
(newline)

(0 2 8 34)


In [3]:
(map fib (list 0 1 2 3 4 5 6 7 8 9 10))

(0 1 1 2 3 5 8 13 21 34 55)

上記2つの処理は、一見全く異なるものに見えますが、処理を分割すると共通性が見えてきます。

- sum-odd-squares手続き
    - 木の葉を列挙 
    - フィルタによって奇数を選ぶ（フィルタ）
    - 選ばれた数の⼆乗を求める（マップ）
    - +を使って、0から始めて結果を合計する（集積）
- even-fibs手続き
    - 0からnまでの数値を列挙 
    - それぞれの整数に対するフィボナッチ数を求る（マップ）
    - フィルタによって偶数を選ぶ（フィルタ）
    - consを使って、空リストから始めて結果をリストにする（集積）

#### 列の演算

ここでは、以下を実装します。
- フィルタ
- 集積
- 木の葉を列挙
- 0からnまでの数値を列挙

マップは定義済みのものを使用します。  
列挙はプログラム毎に実装します。（ただし、出力はリスト）

<img src="2.7.png" width="100%">

<div style="text-align: center;">図2.7:⼿続きsum-odd-squares(上)とeven-fibs(下)を信号の流れという図式によって表現すると、⼆つのプログラ ムの共通性が明らかになる。</div>

In [4]:
; フィルタ
(define (filter predicate sequence)
  (cond ((null? sequence) '())
        (
         (predicate (car sequence))
             (cons (car sequence) (filter predicate (cdr sequence)))
         )
        (else (filter predicate (cdr sequence)))) ; 条件を満たさない要素は無視する
  )

; 動作確認
(display (filter odd? (list 1 2 3 4 5 6 7 8 9 10)))
(newline)
(display (filter even? (list 1 2 3 4 5 6 7 8 9 10)))
(newline)

(1 3 5 7 9)
(2 4 6 8 10)


In [5]:
; 集積
(define (accumulate op initial sequence)
  (if (null? sequence) initial
      (op (car sequence) (accumulate op initial (cdr sequence))))) ; cdrダウンで各要素についてopの処理を施す

; 動作確認
(display (accumulate + 0 (list 1 2 3 4 5)))
(newline)
(display (accumulate * 1 (list 1 2 3 4 5)))
(newline)
(display (accumulate cons '() (list 1 2 3 4 5)))
(newline)

15
120
(1 2 3 4 5)


In [6]:
; 整数列の列挙
(define (enumerate-interval low high)
  (if (> low high) '()
      (cons low (enumerate-interval (+ low 1) high))))

(enumerate-interval 2 7)

(2 3 4 5 6 7)

In [7]:
; 木の葉の列挙
; 葉の要素をリストに変換する。
; 練習問題2.28のfringe手続きそのものであることに注意。
(define (enumerate-tree tree)
  (cond ((null? tree) '())
        ((not (pair? tree)) (list tree))
        (else (append (enumerate-tree (car tree)) (enumerate-tree (cdr tree)))))
  )

(enumerate-tree (list 1 (list 2 (list 3 4)) 5))

(1 2 3 4 5)

In [8]:
; sum-odd-squares手続きを共通化したモジュールを使って実装した場合
(define (sum-odd-squares tree)
  (accumulate + 0 (map square (filter odd? (enumerate-tree tree))))
  )

(sum-odd-squares (list 1 (list 2 (list 3 4) 5) (list 6 7)))

84

In [9]:
; even-fibs手続きを共通化したモジュールを使って実装した場合
(define (even-fibs n)
  (accumulate cons '() (filter even? (map fib (enumerate-interval 0 n)))))

(even-fibs 10)

(0 2 8 34)

In [10]:
; 共通化したモジュールを使用することで、
; フィボナッチ数の2乗の列挙するプログラムも定義済みの手続きの組み合わせで実装ができる。
(define (list-fib-squares n)
  (accumulate cons '() (map square (map fib (enumerate-interval 0 n)))))

(display (map fib '(0 1 2 3 4 5 6 7 8 9 10)))
(newline)
(display (list-fib-squares 10))
(newline)

(0 1 1 2 3 5 8 13 21 34 55)
(0 1 1 4 9 25 64 169 441 1156 3025)


In [11]:
; 奇数の値の2乗の値の積も
; 定義済みの手続きの組み合わせで実装ができる。
(define (product-of-squares-of-odd-elements sequence)
  (accumulate * 1 (map square (filter odd? sequence))))

(product-of-squares-of-odd-elements (list 1 2 3 4 5))

225

⼀般的なデータ処理アプリケーションを列の演算として定式化することもできます。  
人事記録（レコード）の列があるとして、最も給料の⾼いプログラマの給料を見つけたいとします。

- セレクタsalary：人事レコードに含まれる給料を返す
- セレクタprogrammer?：人事レコードがプログラマのものであるかをチェックする

というセレクタが用意されているとすると、  
「最も給料の⾼いプログラマの給料を見つける」というプログラムは次のように書くことができるでしょう。

    (define (salary-of-highest-paid-programmer records) 
      accumulate max 0 (map salary (filter programmer? records))))
      
※ここではあまり深く説明しません。

入出力インターフェースとして、リストを使うことによって、  
各処理モジュールを接続できるようになるので有用です。  
そのため、リストは処理モジュールを接続する標準インターフェイスとして使うことができます。  
（「接続」というのは、手続きの引数に手続きの呼び出しを書くこと。手続きの呼び出しがネストしている）
 
また、構造をリストとして統一することで、  
リストに対する演算と、  
データ構造に依存している処理とを分けて設計することができます。  
これによって、 プログラムの全体的な設計に手を加えずに、  
列の表現⽅法をいろいろ試してみることができます。  
リストを標準インターフェイスとして使うことは、  
3.5節ストリームにの話題につながっていきます。  

#### 練習問題

- [練習問題2.33 accumulateを使ったmap/append/lengthの実装](../exercises/2.33.ipynb)
- [練習問題2.34 ホーナー法](../exercises/2.34.ipynb)
- [練習問題2.35 count-leaves](../exercises/2.35.ipynb)
- [練習問題2.36 accumulate-n](../exercises/2.36.ipynb)
- [練習問題2.37 行列演算](../exercises/2.37.ipynb)
- [練習問題2.38 fold-rightとfold-left](../exercises/2.38.ipynb)
- [練習問題2.39 fold-rightとfold-leftを使ったreverseの実装](../exercises/2.39.ipynb)

#### マップのネスト
列というパラダイムは、ネストしたループによって表現される処理に適用することができます。 　
次の問題について考えてみます。  
「正の整数$n$が与えられたとき、$1 \leq j < i \leq n$で、かつ$i + j$が素数となるような異なる正の整数$i$と$j$のすべての順序つきペアを⾒つけよ。」  
例えば、$n$が$6$のとき、ペアは以下のようになります。 

$$
\begin{array}
{l|ccccccc}
i & 2 & 3 & 4 & 4 & 5 & 6 & 6 \\
j & 1 & 2 & 1 & 3 & 2 & 1 & 5 \\ \hline
i+j & 3 & 5 & 5 & 7 & 7 & 7 & 11 
\end{array}
$$

この計算の実装を考えてみます。  
- $n$以下の正の整数からなる、大きい順に並んだすべてのペアの列を生成する。  
- フィルタによって合計が素数となるペアを選択する  
- フィルタを通過したそれぞれの$(i,j)$のペアに対して$(i,j,i+j)$ という三つ組を作る。

ペアの列の生成は以下の方法で出来ます。
- すべての整数$i \leq n$に対して整数$i$を列挙する。  
  -> (enumerate-interval 1 n)で列挙する。
- このような$i$に対して$j < i$となる$j$を列挙し、この$i,j$からペア$(i,j)$を生成する。  
  -> (enumerate-interval 1 (- i 1))で列挙し、(list i j)を列挙する。
- すべての$i$に対して、すべての列を(appendで集積して)組み合わせることで、求めるペアの列を生成する。

これによって、それぞれの$i$に対するペアの列ができます。 

In [12]:
; ペア(i, j)の列の生成
(define n 6)
(accumulate append '()
            (map (lambda (i)
                   (map (lambda (j) (list i j))
                        (enumerate-interval 1 (- i 1)))
                   )
                 (enumerate-interval 1 n)
                 )
    )

((2 1) (3 1) (3 2) (4 1) (4 2) (4 3) (5 1) (5 2) (5 3) (5 4) (6 1) (6 2) (6 3) (6 4) (6 5))

In [13]:
; マップと集積をappendによって組み合わせる処理はよく使われるので、
; 独立した⼿続きとして実装する。
(define (flatmap proc seq)
  (accumulate append '() (map proc seq)))

In [14]:
; フィルタの述語。
; ペアの合計が素数かどうか。
(define (prime-sum? pair)
  (prime? (+ (car pair) (cadr pair))))

(define (smallest-divisor n)
  (find-divisor n 2)
  )
(define (find-divisor n test-divisor)
  (cond ((> (square test-divisor) n) n)
        ((divides? test-divisor n) test-divisor)
        (else (find-divisor n (+ test-divisor 1)))
    )
  )
(define (divides? a b)
  (= (remainder b a) 0)
  )
(define (prime? n)
  (= n (smallest-divisor n))
  )

In [15]:
; フィルタを通ったペアの列に対して次の⼿続きでマップして、結果の列を⽣成する。
; ペアの⼆つの要素とその合計からなる三つ組(i, j , i + j)を構築する。 
(define (make-pair-sum pair)
  (list (car pair) (cadr pair) (+ (car pair) (cadr pair))))

In [16]:
; 上記手続きを組み合わせて完成した手続き
(define (prime-sum-pairs n)
  (map make-pair-sum (filter prime-sum?
                             (flatmap (lambda (i)
                                        (map (lambda (j) (list i j))
                                             (enumerate-interval 1 (- i 1)))
                                        )
                                (enumerate-interval 1 n)
                               )
                        )
       )
  )

; 動作確認
(prime-sum-pairs 6)

((2 1 3) (3 2 5) (4 1 5) (4 3 7) (5 2 7) (6 1 7) (6 5 11))

マップのネストは、ある集合$S$に対する順列の列挙にも役に立ちます。  

$S=\{1,2,3\}$ である場合、順列は以下のようになります。  
$\{1,2,3\}, \{1,3,2\}, \{2,1,3\}, \{2,3,1\}, \{3,1,2\}, \{3,2,1\}$  

集合$S$の順列を生成する方法として、以下の方法が使えます。  
$S$の各要素$x$に対して、$S - \{x\}$の順列を生成し、それぞれの先頭に$x$を追加する。  

$S - \{x\}$の順列を生成して、生成した順列のそれぞれの先頭に$x$を追加することで、  
$x$から始まる$S$の順列が得られます。  
これを全ての$x$について行うので、$S$の順列がすべて得られます。

In [17]:
; 順列
; 練習問題2.32の回答に近いことに注意。
(define (permutations s)
  (if (null? s) (list '()) ;集合sは空だったら空集合を持つ列
      (flatmap (lambda (x) (map (lambda (p) (cons x p))
                                (permutations (remove x s))))
               s)
      )
  )

In [18]:
; 与えられたリストから指定された要素を除いたリストを返す。
(define (remove item sequence)
  (filter (lambda (x) (not (= x item))) sequence))

; 動作確認
(remove 3 '(1 2 3 4 5 6 5 4 3 2 1))

(1 2 4 5 6 5 4 2 1)

In [19]:
; 動作確認
(permutations '(1 2 3))

((1 2 3) (1 3 2) (2 1 3) (2 3 1) (3 1 2) (3 2 1))

In [20]:
(define (permutations-debug s)
  (if (null? s) (list '()) ;集合sは空だったら空集合を持つ列
      (flatmap (lambda (x) (map
                            (lambda (p)
                              (display x)
                              (display ",")
                              (display p)
                              (display ",")
                              (display (cons x p))
                              (newline)
                              (cons x p)
                              ) (permutations-debug (remove x s))))
               s)
      )
  )

; 動作確認
(permutations-debug '(1 2 3))

3,(),(3)
2,(3),(2 3)
2,(),(2)
3,(2),(3 2)
1,(2 3),(1 2 3)
1,(3 2),(1 3 2)
3,(),(3)
1,(3),(1 3)
1,(),(1)
3,(1),(3 1)
2,(1 3),(2 1 3)
2,(3 1),(2 3 1)
2,(),(2)
1,(2),(1 2)
1,(),(1)
2,(1),(2 1)
3,(1 2),(3 1 2)
3,(2 1),(3 2 1)


((1 2 3) (1 3 2) (2 1 3) (2 3 1) (3 1 2) (3 2 1))

s=(1 2 3)の順序列挙方法。  

    (1 2 3)  
    ->x=1 s=(2 3)  
    ->    x=2 s=(3) ->(2 3)
    ->    x=3 s=(2) ->(3 2)
    ->    ((2 3) (3 2))
    -> ((1 2 3) (1 3 2)

    ->x=2 s=(1 3)  
    ->    x=1 s=(3) ->(1 3)
    ->    x=3 s=(1) ->(3 1)
    ->    ((1 3) (3 1))
    -> ((2 1 3) (2 3 1)

    ->x=3 s=(1 2)  
    ->    x=1 s=(2) ->(1 2)
    ->    x=2 s=(1) ->(2 1)
    ->    ((1 2) (2 1))
    -> ((3 1 2) (3 2 1)

In [21]:
; (i, j)の列挙
(define (enum-nn n)
    (accumulate cons '()
            (map (lambda (i)
                   (accumulate cons '() (map (lambda (j) (list i j))
                        (enumerate-interval 1 n))
                   )
                   )
                 (enumerate-interval 1 n)
                 )
    )
  )
(enum-nn 4)

(((1 1) (1 2) (1 3) (1 4)) ((2 1) (2 2) (2 3) (2 4)) ((3 1) (3 2) (3 3) (3 4)) ((4 1) (4 2) (4 3) (4 4)))

In [22]:
; ネストしたループの動作確認
(define (enum-nnn n)
  (define (iter k)
    (if (= k 0) (list ())
        (filter
         (lambda (x) #t)
          (flatmap (lambda (rest)
                    (map (lambda (new-row) (append rest (list new-row)))
                         (enumerate-interval 1 n))
                    )
                  (iter (- k 1))
          )
         )
        )
    )
  (iter n)
  )
(enum-nnn 4)

((1 1 1 1) (1 1 1 2) (1 1 1 3) (1 1 1 4) (1 1 2 1) (1 1 2 2) (1 1 2 3) (1 1 2 4) (1 1 3 1) (1 1 3 2) (1 1 3 3) (1 1 3 4) (1 1 4 1) (1 1 4 2) (1 1 4 3) (1 1 4 4) (1 2 1 1) (1 2 1 2) (1 2 1 3) (1 2 1 4) (1 2 2 1) (1 2 2 2) (1 2 2 3) (1 2 2 4) (1 2 3 1) (1 2 3 2) (1 2 3 3) (1 2 3 4) (1 2 4 1) (1 2 4 2) (1 2 4 3) (1 2 4 4) (1 3 1 1) (1 3 1 2) (1 3 1 3) (1 3 1 4) (1 3 2 1) (1 3 2 2) (1 3 2 3) (1 3 2 4) (1 3 3 1) (1 3 3 2) (1 3 3 3) (1 3 3 4) (1 3 4 1) (1 3 4 2) (1 3 4 3) (1 3 4 4) (1 4 1 1) (1 4 1 2) (1 4 1 3) (1 4 1 4) (1 4 2 1) (1 4 2 2) (1 4 2 3) (1 4 2 4) (1 4 3 1) (1 4 3 2) (1 4 3 3) (1 4 3 4) (1 4 4 1) (1 4 4 2) (1 4 4 3) (1 4 4 4) (2 1 1 1) (2 1 1 2) (2 1 1 3) (2 1 1 4) (2 1 2 1) (2 1 2 2) (2 1 2 3) (2 1 2 4) (2 1 3 1) (2 1 3 2) (2 1 3 3) (2 1 3 4) (2 1 4 1) (2 1 4 2) (2 1 4 3) (2 1 4 4) (2 2 1 1) (2 2 1 2) (2 2 1 3) (2 2 1 4) (2 2 2 1) (2 2 2 2) (2 2 2 3) (2 2 2 4) (2 2 3 1) (2 2 3 2) (2 2 3 3) (2 2 3 4) (2 2 4 1) (2 2 4 2) (2 2 4 3) (2 2 4 4) (2 3 1 1) (2 3 1 2) (2 3 1 3) (2 3 1 4)

#### 練習問題

- [練習問題2.40 unique-pairs/prime-sum-pairs](../exercises/2.40.ipynb)
- [練習問題2.41 3変数のマップ](../exercises/2.41.ipynb)
- [練習問題2.42 8クイーンパズル](../exercises/2.42.ipynb)
- [練習問題2.43 間違った8クイーンパズルの回答](../exercises/2.43.ipynb)